In [17]:
# Imports
from modules import adquire, cleaning, create_df_checking, outputs


def main(): 
    df_students, df_katas = adquire()    
    df_students, df_katas = cleaning(df_students, df_katas)
    df,hora= create_df_checking(df_students, df_katas)    
    outputs(df,False)
    return df


if __name__ == '__main__':
    df = main()

gontzalm presentará la kata street-fighter-2-character-selection


slug,pete-the-baker,street-fighter-2-character-selection
username,,
VanessaMacC,False,False
silviaherf,True,True
rfminguez,True,False
PaulaNuno,False,False
lagarto,True,False
Marta Zavala de Diego,False,False
llarrad,False,False
KevsDe,True,False
JAMS23,True,False
